In [ ]:
import pandas as pd
from dataAnalysis.DataAnalysis import DataAnalysis

data = pd.read_csv(r"extdata/sbcdata.csv", header=0)
mimc_data = pd.read_csv("mimic_cbc.csv")
data_analysis = DataAnalysis(data, mimc_data)
mimic_data = data_analysis.get_mimic_data()

In [ ]:
labs = pd.read_csv("../mimic/mimic-iv-2.2/hosp/labevents.csv")

In [ ]:
micro_events = pd.read_csv("../mimic/mimic-iv-2.2/hosp/microbiologyevents.csv")

In [ ]:
omr_events = pd.read_csv("../mimic/mimic-iv-2.2/hosp/omr.csv")
omr_pivot = omr_events.pivot_table(index=["subject_id",  "chartdate"],columns='result_name', values="result_value", aggfunc='first').reset_index()

In [ ]:
omr_item_ids = []
nan_values = []
for i, column in enumerate(omr_pivot.columns):
    if i < 2: 
        continue
    omr_item_ids.append(column)
    nan_values.append(omr_pivot[column].isna().sum())
    print(f"{column} - {str(omr_pivot[column].isna().sum())}")
omr_df = pd.DataFrame({"omr_item_ids": omr_item_ids, "nan_values": nan_values})
omr_df.sort_values("nan_values").reset_index().iloc[:8, :]["omr_item_ids"].values

In [ ]:
pivot_hemato_df_wo_hadm = labs.pivot_table(index=["subject_id",  "charttime"],columns='itemid', values="valuenum", aggfunc='first').reset_index()
# hemato_df_pivot = pd.merge(pivot_hemato_df_wo_hadm, hemato_df[["subject_id", "charttime", "hadm_id"]].drop_duplicates(), on=["subject_id", "charttime"], how="left")

In [ ]:
pivot_hemato_df_wo_hadm.pop(51301)
pivot_hemato_df_wo_hadm.pop(51279)
pivot_hemato_df_wo_hadm.pop(51265)
pivot_hemato_df_wo_hadm.pop(51222)
pivot_hemato_df_wo_hadm.pop(51250)

In [ ]:
lab_item_ids = []
nan_values = []
for i, column in enumerate(pivot_hemato_df_wo_hadm.columns):
    if i < 2: 
        continue
    lab_item_ids.append(column)
    nan_values.append(pivot_hemato_df_wo_hadm[column].isna().sum())
    print(f"{column} - {str(pivot_hemato_df_wo_hadm[column].isna().sum())}")

lab_nan_df = pd.DataFrame(data={"lab_item_ids": lab_item_ids, "nan_values": nan_values})

In [ ]:
import numpy as np
most_prev_lab_items = np.squeeze(lab_nan_df.sort_values("nan_values").iloc[:20, :].loc[:, ["lab_item_ids"]].values)

In [ ]:
most_prev_lab_items

In [ ]:
list(map(lambda x: f"F{str(x[0]+1)}", enumerate(most_prev_lab_items.tolist())))

In [ ]:
more_labs = pd.merge(mimic_data, pivot_hemato_df_wo_hadm.loc[:, ["subject_id","charttime", 50970]] , on="subject_id")

In [ ]:
more_labs['outtime']= pd.to_datetime(more_labs['outtime']).dt.tz_localize(None)
more_labs['intime']= pd.to_datetime(more_labs['intime']).dt.tz_localize(None)
more_labs['charttime_y']= pd.to_datetime(more_labs['charttime_y']).dt.tz_localize(None)

In [ ]:
more_labs.info()

In [ ]:
more_labs_red = more_labs[more_labs["charttime_y"] >= more_labs["intime"]]
more_labs_red = more_labs[more_labs["charttime_y"] < more_labs["outtime"]]

In [ ]:
more_labs_red_unique = more_labs_red.drop_duplicates(subset=["subject_id", "HGB", "RBC", "WBC", "CRP", "PLT", "MCV", 50970, "Age", "Sex", "Label"])

In [ ]:
more_labs_red_num = more_labs_red_unique.dropna(subset=["HGB", "RBC", "WBC", "CRP", "PLT", "MCV", "Age", "Sex"])

In [ ]:
more_labs_red_num

In [ ]:
labs.head()

In [ ]:
labs.columns